In [1]:
from dataclasses import dataclass
from enum import Enum
from pathlib import Path
from typing import List

import matplotlib.pyplot as plt
import numpy as np

from gps_accuracy.gps_accuracy import GpxResult
%load_ext autoreload
%autoreload 2

In [91]:
from gps_accuracy.gps_accuracy import GpxEvaluator
import pandas as pd

class InputType(Enum):
    Touch = 1
    TUI = 2
    
class Metaphor(Enum):
    Gesture = 1
    Joystick = 2
    Car =  3
    
@dataclass
class ReferenceTrack:
    track_id: int
    file: Path
    
    def __init__(self, file_path: Path):
        self.track_id = int(file_path.stem)
        self.file = file_path

@dataclass
class RecordedTrack:
    track_id: int
    user_id: int
    input_type: InputType
    metaphor: Metaphor
    file: Path
    result: GpxResult
    
    def __init__(self, file_path: Path):
        file_name = file_path.stem
        parts = file_name.split("_")
        self.track_id: int = int(parts[0])
        self.user_id: int = int(parts[1])
        self.input_type: InputType = InputType[parts[2]]
        self.metaphor: Metaphor = Metaphor[parts[3]]
        self.file: Path = file_path
    
    def evaluate(self, reference_track: ReferenceTrack):
        evaluator = GpxEvaluator(reference_track.file, self.file)
        self.result = evaluator.evaluate()
              
@dataclass
class TrackRepository:
    reference_tracks: dict
    recorded_tracks: List[RecordedTrack]
    
    def __init__(self):
        reference_track_list = [ReferenceTrack(track_file) for track_file in Path("reference_tracks").iterdir() if track_file.is_file()]
        self.reference_tracks = {track.track_id: track for track in reference_track_list}
        self.recorded_tracks = [RecordedTrack(track_file) for track_file in Path("recorded_tracks").iterdir() if track_file.is_file()]
        self._evaluate() 
    
    def _evaluate(self):
        for track in self.recorded_tracks:
            reference_track = self.reference_tracks[track.track_id]
            track.evaluate(reference_track)
        
    def get_by_track(self, track_id: int) -> List[RecordedTrack]:
        return [track for track in self.recorded_tracks if track.track_id == track_id]

    def get_by_user(self, user_id: int) -> List[RecordedTrack]:
        return [track for track in self.recorded_tracks if track.user_id == user_id]
    
    def get_by_input_type(self, input_type: InputType) -> List[RecordedTrack]:
        return [track for track in self.recorded_tracks if track.input_type == input_type]
    
    def get_by_metaphor(self, metaphor: Metaphor) -> List[RecordedTrack]:
        return [track for track in self.recorded_tracks if track.metaphor == metaphor]
    
    def get_all(self) -> List[RecordedTrack]:
        return self.recorded_tracks

class InputFilter(Enum):
    InputAll = 1
    InputCategorized = 2
    
class ResultParam(Enum):
    Time = 1
    MeanError = 2
    MedianError = 3
    PercentileError = 4

class ResultPlotter:
    def __init__(self):
        self.repo = TrackRepository()
        tracks = self.repo.get_all()
        data = {
            'Track':  [track.track_id for track in tracks],
            'UserId': [track.user_id for track in tracks],
            'InputAll': [f"{track.input_type.name}_{track.metaphor.name}" for track in tracks],
            'InputCategorized': [track.input_type.name for track in tracks],
            'Time': [track.result.time for track in tracks],
            'MeanError': [track.result.mean for track in tracks],
            'MedianError': [track.result.median for track in tracks],
            'PercentileError': [track.result.percentile for track in tracks]
        }
        self.data_frame = pd.DataFrame(data)
        self.data_frame.style.format(precision=2, decimal=".")
    
    def summary(self):
        return self.data_frame.style.format(precision=2)
    
    def print_result(self, result_param: ResultParam, input_filter: InputFilter, aggfunc: str, plot = False, color=False):
        table = self.data_frame.pivot_table(index=input_filter.name, columns="Track", values=result_param.name, aggfunc=[aggfunc], sort=False)
        if plot:
            plt.figure()
            table.plot.bar()
        style = table.style
        if color:
            style = style.background_gradient(axis=0, cmap='Reds')
        return style.format(precision=2)

    

In [92]:
plotter = ResultPlotter()

In [65]:
plotter.summary()

,Track,UserId,InputAll,InputCategorized,Time,MeanError,MedianError,PercentileError
0,1,1,Touch_Gesture,Touch,55.00,1.68,1.18,5.07
1,1,1,Touch_Joystick,Touch,42.00,1.72,1.15,5.17
2,1,1,TUI_Car,TUI,56.00,1.44,1.05,4.42
3,1,1,TUI_Joystick,TUI,41.00,3.93,2.13,13.07
4,1,2,Touch_Gesture,Touch,41.00,1.83,1.49,4.76
5,1,2,Touch_Joystick,Touch,39.00,1.11,0.80,3.58
6,1,2,TUI_Car,TUI,45.00,3.52,1.84,23.14
7,1,2,TUI_Joystick,TUI,40.00,2.95,1.79,8.44
8,2,1,Touch_Gesture,Touch,53.00,2.63,2.25,6.63
9,2,1,Touch_Joystick,Touch,47.00,2.13,1.63,6.18


In [93]:
plotter.print_result(ResultParam.Time, InputFilter.InputAll, aggfunc='mean', color=True, plot=False) # Time in seconds

In [94]:
plotter.print_result(ResultParam.Time, InputFilter.InputAll, aggfunc='std', plot=False)

In [95]:
plotter.print_result(ResultParam.MeanError, InputFilter.InputAll, aggfunc='mean', color=True, plot=False) # Mean Error in meter